In [30]:
from arcgis.gis import GIS
import pandas as pd
from copy import deepcopy
from ARCGIS_PASSWORD import USERNAME, PASSWORD, ARCGIS_URL
import datetime

from CONSTANTS import ARCGIS_TIME_FORMAT

# name of the feature layer to be modified
FEATURE_LAYER_NAME = 'interval_1hour_feature_layer'

# csv source for the data to be added to the feature layer
CSV_INPUT_SOURCE = 'processed_data/interval_1hour_All.csv'

# check for whether old data before a certain point in time should be deleted from the feature layer
# and how old should the data be to be deleted from the feature layer (in days or weeks, preferably only one
# should be used: 10 days 0 weeks / 0 days 5 weeks)
# e.g. delete data older than two weeks etc...
REMOVE_OLD_FEATURE_LAYER_DATA = True
REMOVE_OLD_FEATURE_LAYER_DATA__DAYS = 0
REMOVE_OLD_FEATURE_LAYER_DATA__WEEKS = 2 

time_now = datetime.datetime.now()
remove_data_before = time_now - datetime.timedelta(
    days=REMOVE_OLD_FEATURE_LAYER_DATA__DAYS,
    weeks=REMOVE_OLD_FEATURE_LAYER_DATA__WEEKS,                        
    hours= time_now.hour,                         
    minutes=time_now.minute,      
    seconds=time_now.second,         
    microseconds=time_now.microsecond
)
remove_data_before = pd.Timestamp(remove_data_before)
remove_data_before

Timestamp('2022-10-28 00:00:00')

In [2]:
# Logging in to ArcGIS
print('Logging in to ArcGIS')
gis = GIS(ARCGIS_URL, USERNAME, PASSWORD)
print('Successfully logged in')


Logging in to ArcGIS
Successfully logged in


In [13]:
# Read the data to be added to the featured layer from csv file
print('Reading CSV data')
csv_data = pd.read_csv(CSV_INPUT_SOURCE)
old_columns = list(csv_data.columns)
columns_replace = {}
for i in old_columns:
    columns_replace[i] = i.strip().replace(' ', '_').replace('.', '_')
csv_data = csv_data.rename(columns = columns_replace)



Reading CSV data


In [14]:
# Search for all content by the user - this part of the code is just to check which items are found, this part is redundant and can be removed
ArcGIS_content_search = gis.content.search('owner:' + USERNAME)
ArcGIS_content_search

[<Item title:"interval_1hour_feature_layer" type:CSV owner:nuscde_bdpt>,
 <Item title:"Station Data" type:Web Map owner:nuscde_bdpt>,
 <Item title:"interval_1hour_feature_layer" type:Feature Layer Collection owner:nuscde_bdpt>,
 <Item title:"Station 1hr data dashboard" type:Dashboard owner:nuscde_bdpt>]

In [18]:
# get the feature layer to be updated
print('Getting Feature Layer')
interval_data_search = gis.content.search('owner:' + USERNAME + ' AND title:' + FEATURE_LAYER_NAME, item_type='Feature Layer Collection')
interval_data_item = interval_data_search[0]
interval_data_f_layer = interval_data_item.layers[0]
interval_data_f_set = interval_data_f_layer.query() #querying without any conditions returns all the features
interval_data_f_set.sdf.head()

Getting Feature Layer


,Station,Serial_number,Latitude,Longitude,Floor,Height,Location_description,Building_name,Timestamp,Wind_Speed,Temperature_PT100,Relative_Humidity,Solar_Radiation,PM1_0,PM2_5,PM10,ObjectId,SHAPE
0,#1,21004880,1.277144,103.838944,Ground,2.5m,"lamppost 7/4, between BLK 3 and Blk 7 in Evert...",Everton park,2022-10-28 00:00:00,0.373333,27.453333,78.483333,0.066667,5.873333,8.090000,11.566667,3251,"{""x"": 103.838944444444, ""y"": 1.27714444444444,..."
1,#1,21004880,1.277144,103.838944,Ground,2.5m,"lamppost 7/4, between BLK 3 and Blk 7 in Evert...",Everton park,2022-10-28 01:00:00,0.433333,26.070000,85.013333,0.366667,5.203333,7.543333,11.230000,3252,"{""x"": 103.838944444444, ""y"": 1.27714444444444,..."
2,#1,21004880,1.277144,103.838944,Ground,2.5m,"lamppost 7/4, between BLK 3 and Blk 7 in Evert...",Everton park,2022-10-28 02:00:00,0.440000,25.926667,85.393333,0.033333,5.583333,8.153333,11.033333,3253,"{""x"": 103.838944444444, ""y"": 1.27714444444444,..."
3,#1,21004880,1.277144,103.838944,Ground,2.5m,"lamppost 7/4, between BLK 3 and Blk 7 in Evert...",Everton park,2022-10-28 03:00:00,0.333333,25.906667,85.846667,0.400000,5.510000,7.913333,10.623333,3254,"{""x"": 103.838944444444, ""y"": 1.27714444444444,..."
4,#1,21004880,1.277144,103.838944,Ground,2.5m,"lamppost 7/4, between BLK 3 and Blk 7 in Evert...",Everton park,2022-10-28 04:00:00,0.400000,26.186667,83.900000,1.633333,5.846667,8.170000,11.370000,3255,"{""x"": 103.838944444444, ""y"": 1.27714444444444,..."


In [29]:
# if the check for removing data is set to true, make a list of data to be removed (features_to_be_removed)
# regardless of the removing data check, make a dictionary of the existing stations SHAPE ({x: longitude_value, y: latitude_value}) 
# as well as existing timestamp (so that we can check for and avoid adding duplicate data later on)
print('Process data to be removed')
current_data = interval_data_f_set.sdf.to_dict('records')
features_to_be_removed = []

station_SHAPE = {}
existing_timestamps = set()
for i in current_data:
    if REMOVE_OLD_FEATURE_LAYER_DATA and (i['Timestamp'] < remove_data_before):
        features_to_be_removed.append(i['ObjectId'])
    existing_timestamps.add(i['Timestamp'].strftime(ARCGIS_TIME_FORMAT))
    if i['Station'] in station_SHAPE:
        continue
    station_SHAPE[i['Station']] = i['SHAPE']
print(features_to_be_removed.__len__())


Process data to be removed
3192


In [7]:
# convert the csv data to dictionary
new_data = csv_data.to_dict('records')
new_data[0]


{'Station': '#1',
 'Serial_number': 21004880,
 'Latitude': 1.27714444444444,
 'Longitude': 103.838944444444,
 'Floor': 'Ground',
 'Height': '2.5m',
 'Location_description': 'lamppost 7/4, between BLK 3 and Blk 7 in Everton Park',
 'Building_name': 'Everton park',
 'Timestamp': '2022/11/04 11:00:00',
 'Wind_Speed': 0.7066666666666668,
 'Temperature': 30.409999999999997,
 'Relative_Humidity': 67.64,
 'Solar_Radiation': 387.5,
 'PM1_0': 5.336666666666667,
 'PM2_5': 7.856666666666666,
 'PM10': 11.56333333333333}

In [8]:
# check for any difference in field names - redundant code, can be removed
new_field_names = list(interval_data_f_set.sdf.columns.difference(csv_data.columns))
new_field_names

['ObjectId', 'SHAPE', 'Temperature_PT100']

In [9]:
# make a list of the data that are to be added to the feature layer,
# the new added data should be in this format: 
#   {
#       'geometry': {'x': longitude_value, 'y': latitude_value},
#       'attribute': {
#           'Station': '#xx',
#           'Serial_number': xxxxxxxx,
#           'Latitude': x.xxxx,
#           'Longitude': xxx.xxx,
#           'Floor': 'xxxx',
#           'Height': 'xxxx',
#           'Location_description': 'xxxxxxxxxxxx',
#           'Building_name': 'xxxxxxxx',
#           'Timestamp': 'xxxxxxxxxxxx',
#           'Wind_Speed': x.xxxx,
#           'Temperature': x.xxxx,
#           'Relative_Humidity': x.xxxx,
#           'Solar_Radiation': x.xxxx,
#           'PM1_0': x.xxxx,
#           'PM2_5': x.xxxx,
#           'PM10': x.xxxx
#       } 
#   } 
features_to_be_added = []
for data_row in new_data:
    if data_row['Timestamp'] in existing_timestamps:
        continue
    new_added_row = {
        "geometry": {
            'x': station_SHAPE[data_row['Station']]['x'],
            'y': station_SHAPE[data_row['Station']]['y']
        },
        "attributes": data_row
    }
    features_to_be_added.append(new_added_row)
features_to_be_added.__len__()

3097

In [10]:
# update the feature layer, adding features_to_be_added and deleting features_to_be_removed
interval_data_f_layer.edit_features(adds = features_to_be_added, deletes = features_to_be_removed)

{'addResults': [{'objectId': 7791,
   'uniqueId': 7791,
   'globalId': None,
   'success': True},
  {'objectId': 7792, 'uniqueId': 7792, 'globalId': None, 'success': True},
  {'objectId': 7793, 'uniqueId': 7793, 'globalId': None, 'success': True},
  {'objectId': 7794, 'uniqueId': 7794, 'globalId': None, 'success': True},
  {'objectId': 7795, 'uniqueId': 7795, 'globalId': None, 'success': True},
  {'objectId': 7796, 'uniqueId': 7796, 'globalId': None, 'success': True},
  {'objectId': 7797, 'uniqueId': 7797, 'globalId': None, 'success': True},
  {'objectId': 7798, 'uniqueId': 7798, 'globalId': None, 'success': True},
  {'objectId': 7799, 'uniqueId': 7799, 'globalId': None, 'success': True},
  {'objectId': 7800, 'uniqueId': 7800, 'globalId': None, 'success': True},
  {'objectId': 7801, 'uniqueId': 7801, 'globalId': None, 'success': True},
  {'objectId': 7802, 'uniqueId': 7802, 'globalId': None, 'success': True},
  {'objectId': 7803, 'uniqueId': 7803, 'globalId': None, 'success': True},
  